In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import optuna
import joblib
from collections import namedtuple

In [2]:
Data = namedtuple('Data', ["type","name", "path", "config"])

In [3]:
DATA_PATH = Path("data")
OUTPUT_PATH = Path("output")
STUDY_PATH = Path("study")

In [4]:
POL_CLS_UUID = "66dc899c4c4f4e4f822d02bdddb18f80"

In [5]:
experiments = [
 'classification|default-of-credit-card-clients',
 'classification|heloc',
 'classification|eye_movements',
 'classification|Higgs',
 'classification|pol',
 'classification|albert',
 'classification|road-safety',
 'classification|MiniBooNE',
 'classification|covertype',
 'classification|jannis',
 'classification|Bioresponse',
 'regression|cpu_act',
 'regression|Ailerons',
 'regression|yprop_4_1',
 'regression|superconduct',
 'regression|Allstate_Claims_Severity',
 'regression|topo_2_1',
 'regression|Mercedes_Benz_Greener_Manufacturing',
]

# experiments = [Experiment(*d.split("|")) for d in experiments]

In [6]:
datasets = []
for experiment in experiments:
    typ, name = experiment.split("|")
    folder = DATA_PATH/name
    config_files = list(folder.glob("*config*"))
    config = np.load(config_files[0], allow_pickle=True).item()
    config["n_iter"] = len(config_files)
    datasets.append(Data(typ, name, folder, config))

In [7]:
dataset_dict = {d.name: d for d in datasets}

In [8]:
dataset_names = list(dataset_dict.keys())
dataset_names

['default-of-credit-card-clients',
 'heloc',
 'eye_movements',
 'Higgs',
 'pol',
 'albert',
 'road-safety',
 'MiniBooNE',
 'covertype',
 'jannis',
 'Bioresponse',
 'cpu_act',
 'Ailerons',
 'yprop_4_1',
 'superconduct',
 'Allstate_Claims_Severity',
 'topo_2_1',
 'Mercedes_Benz_Greener_Manufacturing']

In [9]:
sel_columns = [
 'Unnamed: 0',
 'benchmark',
 'data__keyword',
 '_timestamp',
 'model_name',
 'model_type',
 'one_hot_encoder',

 'mean_time',
 'std_time',

 'mean_test_score',

 'max_test_score',
 'min_test_score',
 'std_test_score',

 'test_scores',
 'times',
 ]

In [10]:
experiment_results = pd.DataFrame(columns=sel_columns)

In [11]:
def get_study(dataset_name):
    if dataset_name == "pol":
        return pd.read_csv(OUTPUT_PATH/f"study_pol_{POL_CLS_UUID}.csv")
    folder = OUTPUT_PATH
    outputs = [f for f in folder.glob("study*") if dataset_name in f.name]
    if len(outputs)==1:
        return pd.read_csv(outputs[0])
    else: # For cncelled and unfinished trials
        folder  = STUDY_PATH
        study_path = [f for f in folder.glob(f"*{dataset_name}*.db") if "full" not in f.name][0]
        study_name = study_path.name.split(".")[0]
        study = optuna.load_study(study_name=study_name, storage=f"sqlite:///{study_path}")
        return study.trials_dataframe()
    
def format_study(study, dataset):
    type = dataset_dict[dataset.name].type
    categorical = dataset_dict[dataset.name].config["data__categorical"]
    if categorical==1.0:
        benchmark = "categorical"
    else:
        benchmark = "numerical"
    benchmark+=f"_{type}_medium"
    study['benchmark'] = benchmark
    study["data__keyword"] = dataset.name
    # try:
    study["duration"] = pd.to_timedelta(study["duration"], errors="coerce")
    study["mean_time"] = study["duration"].dt.total_seconds()/dataset_dict[dataset.name].config["n_iter"]
    # except AttributeError as e:
    #     for i, row in study.iterrows():
    #         try:
    #             study.loc[i, "duration"].dt.total_seconds() 
    #         except AttributeError as e:
    #             print(row["duration"])
    #             raise e
    #     raise e
    rename_dict = {
        "datetime_start": "_timestamp",
        "value": "mean_test_score",
    }
    study.rename(columns=rename_dict, inplace=True)
    study["model_name"] = "GFLU"
    study["model_type"] = "PyTorchTabular"
    param_cols = [c for c in study.columns if c.startswith("params_")]
    study['params']=study[param_cols].apply(dict, axis=1)
    study = study.loc[study.state=="COMPLETE"]
    intersection_cols = list(set(study.columns).intersection(set(sel_columns)))
    study = study.loc[:, intersection_cols+["params"]]
    return study
            

In [12]:
#Add row for each experiment
for dataset in datasets:
    print(dataset.name)
    study = get_study(dataset.name)
    study = format_study(study, dataset)
    experiment_results = pd.concat([experiment_results,study], ignore_index=True)

default-of-credit-card-clients
heloc
eye_movements
Higgs
pol
albert
road-safety
MiniBooNE
covertype
jannis
Bioresponse
cpu_act
Ailerons
yprop_4_1
superconduct
Allstate_Claims_Severity
topo_2_1
Mercedes_Benz_Greener_Manufacturing


In [13]:
experiment_results

,Unnamed: 0,benchmark,data__keyword,_timestamp,model_name,model_type,one_hot_encoder,mean_time,std_time,mean_test_score,max_test_score,min_test_score,std_test_score,test_scores,times,params
0,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:41:46.516483,GFLU,PyTorchTabular,NaN,14.715429,NaN,0.503480,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.4753571532049581, 'p..."
1,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:42:30.691539,GFLU,PyTorchTabular,NaN,50.556374,NaN,0.682418,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.15212112147976886, '..."
2,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:45:02.387713,GFLU,PyTorchTabular,NaN,30.633439,NaN,0.721277,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.08526206184364576, '..."
3,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:46:34.307562,GFLU,PyTorchTabular,NaN,95.580769,NaN,0.662469,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.15585553804470548, '..."
4,NaN,categorical_classification_medium,default-of-credit-card-clients,2023-06-09 13:51:21.070666,GFLU,PyTorchTabular,NaN,45.122091,NaN,0.704700,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.13567451588694796, '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,NaN,categorical_regression_medium,Mercedes_Benz_Greener_Manufacturing,2023-06-06 04:21:03.243799,GFLU,PyTorchTabular,NaN,50.869310,NaN,0.592073,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.25374588206967824, '..."
1627,NaN,categorical_regression_medium,Mercedes_Benz_Greener_Manufacturing,2023-06-06 04:25:17.620285,GFLU,PyTorchTabular,NaN,36.404743,NaN,0.594517,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.1803395985027783, 'p..."
1628,NaN,categorical_regression_medium,Mercedes_Benz_Greener_Manufacturing,2023-06-06 04:28:19.667644,GFLU,PyTorchTabular,NaN,88.567702,NaN,0.584889,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.045087026473713876, ..."
1629,NaN,categorical_regression_medium,Mercedes_Benz_Greener_Manufacturing,2023-06-06 04:35:42.535367,GFLU,PyTorchTabular,NaN,154.126833,NaN,0.593898,NaN,NaN,NaN,NaN,NaN,"{'params_gflu_dropout': 0.10281880884483054, '..."


In [14]:
experiment_results["_timestamp"] = pd.to_datetime(experiment_results["_timestamp"])

In [15]:
experiment_results.to_parquet("data/experiment_results.parquet")

## Results Compilation

In [16]:
def calc_best_row(df):
    if df.shape[0] == 0:
        return df
    best_idx = df.mean_test_score.idxmax()
    best_row = df.loc[best_idx]
    best_row["best"] = True
    return best_row
experiment_results = experiment_results.groupby(["benchmark", "data__keyword", "model_name"]).apply(calc_best_row)

In [17]:
experiment_results = experiment_results.loc[:, ["mean_time", "mean_test_score", "test_scores", "times", "params"]]

In [18]:
experiment_results

mean_time  \
benchmark                         data__keyword                       model_name               
categorical_classification_medium albert                              GFLU         57.640220   
                                  covertype                           GFLU        411.813730   
                                  default-of-credit-card-clients      GFLU         35.262454   
                                  eye_movements                       GFLU          8.093059   
                                  road-safety                         GFLU         86.645416   
categorical_regression_medium     Allstate_Claims_Severity            GFLU        172.616248   
                                  Mercedes_Benz_Greener_Manufacturing GFLU        166.836060   
                                  topo_2_1                            GFLU         15.962654   
numerical_classification_medium   Bioresponse                         GFLU          6.701376   
                                  Higgs                               GFLU         65.648203   
                                  MiniBooNE                           GFLU        127.156320   
                                  heloc                               GFLU         16.213848   
                                  jannis                              GFLU          9.166727   
                                  pol                                 GFLU         14.496745   
numerical_regression_medium       Ailerons                            GFLU          8.855283   
                                  cpu_act                             GFLU        157.017723   
                                  superconduct                        GFLU        109.637601   
                                  yprop_4_1                           GFLU          6.395486   

                                                                                  mean_test_score  \
benchmark                         data__keyword                       model_name                    
categorical_classification_medium albert                              GFLU               0.685300   
                                  covertype                           GFLU               0.932200   
                                  default-of-credit-card-clients      GFLU               0.728023   
                                  eye_movements                       GFLU               0.678936   
                                  road-safety                         GFLU               0.808800   
categorical_regression_medium     Allstate_Claims_Severity            GFLU               0.591058   
                                  Mercedes_Benz_Greener_Manufacturing GFLU               0.598374   
                                  topo_2_1                            GFLU               0.048455   
numerical_classification_medium   Bioresponse                         GFLU               0.882231   
                                  Higgs                               GFLU               0.752300   
                                  MiniBooNE                           GFLU               0.929900   
                                  heloc                               GFLU               0.733952   
                                  jannis                              GFLU               0.819800   
                                  pol                                 GFLU               0.991167   
numerical_regression_medium       Ailerons                            GFLU               0.864613   
                                  cpu_act                             GFLU               0.982927   
                                  superconduct                        GFLU               0.925718   
                                  yprop_4_1                           GFLU               0.085295   

                                                                                  test_scores  \
benchmark                         data__keyw

In [19]:
benchmark_results = pd.read_parquet("data/benchmark_best_rows.parquet")

In [20]:
experiment_results = pd.concat([experiment_results, benchmark_results]).sort_values(["benchmark", "data__keyword", "model_name"])

In [21]:
experiment_results.head()

mean_time  \
benchmark                         data__keyword model_name                            
categorical_classification_medium albert        FT Transformer            94.898633   
                                                GFLU                      57.640220   
                                                GradientBoostingTree       3.410833   
                                                HistGradientBoostingTree   2.274791   
                                                MLP                       16.655870   

                                                                          mean_test_score  \
benchmark                         data__keyword model_name                                  
categorical_classification_medium albert        FT Transformer                   0.656275   
                                                GFLU                             0.685300   
                                                GradientBoostingTree             0.657637   
                                                HistGradientBoostingTree         0.657814   
                                                MLP                              0.653226   

                                                                         test_scores  \
benchmark                         data__keyword model_name                             
categorical_classification_medium albert        FT Transformer                  None   
                                                GFLU                             NaN   
                                                GradientBoostingTree            None   
                                                HistGradientBoostingTree        None   
                                                MLP                             None   

                                                                         times  \
benchmark                         data__keyword model_name                       
categorical_classification_medium albert        FT Transformer            None   
                                                GFLU                       NaN   
                                                GradientBoostingTree      None   
                                                HistGradientBoostingTree  None   
                                                MLP                       None   

                                                                                                                     params  
benchmark                         data__keyword model_name                                                                   
categorical_classification_medium albert        FT Transformer                                                          NaN  
                                                GFLU                      {'params_gflu_dropout': 0.014136779525448506, ...  
                                                GradientBoostingTree                                                    NaN  
                                                HistGradientBoostingTree                                                NaN  
                                                MLP                                                                     NaN

In [22]:
experiment_results.to_parquet("data/experiment_results_best_rows.parquet")

In [23]:
test_scores = pd.pivot_table(experiment_results.reset_index(), index=["benchmark", "data__keyword"], columns="model_name", values="mean_test_score")
# format test_scores df so that highest scores is highlighted
test_scores.style.highlight_max(axis=1, color='lightgreen')

In [24]:
mean_times = pd.pivot_table(experiment_results.reset_index(), index=["benchmark", "data__keyword"], columns="model_name", values="mean_time")
# format mean_times df with a red amber green color scale - reversed
mean_times.style.background_gradient(cmap='RdYlGn_r')